# Add various kinds of noise then test denoising methods

In [44]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def load_image_keep_channels(path):
    # preserve number of channels and alpha if present
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(path)
    # convert 4-channel BGRA -> BGR (drop alpha) for metric comparisons
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]
    return img

def to_uint8(img):
    # convert floats to uint8 if needed (assume values in 0..1 for floats)
    if img.dtype == np.float32 or img.dtype == np.float64:
        img = np.clip(img, 0.0, 1.0)
        img = (img * 255).astype(np.uint8)
    else:
        img = img.astype(np.uint8)
    return img

def compute_metrics(ref_path, test_path):
    A = load_image_keep_channels(ref_path)
    B = load_image_keep_channels(test_path)

    # If either is grayscale image read as 2D, keep as 2D. If one is 2D and the other 3D,
    # convert 3D to grayscale to compare apples-to-apples (or replicate channels)
    if A.ndim == 2 and B.ndim == 3:
        B = cv2.cvtColor(B, cv2.COLOR_BGR2GRAY)
    elif A.ndim == 3 and B.ndim == 2:
        A = cv2.cvtColor(A, cv2.COLOR_BGR2GRAY)

    A = to_uint8(A)
    B = to_uint8(B)

    if A.shape != B.shape:
        raise ValueError(f"Shape mismatch: {A.shape} vs {B.shape}")

    # data_range for uint8 images
    data_range = 255 if A.dtype == np.uint8 else (A.max() - A.min())

    # PSNR works for both grayscale and color
    psnr_val = psnr(A, B, data_range=data_range)

    # For SSIM: specify channel_axis for multichannel arrays
    if A.ndim == 3:
        ssim_val = ssim(A, B, data_range=data_range, channel_axis=-1)
    else:
        ssim_val = ssim(A, B, data_range=data_range)

    return psnr_val, ssim_val

In [45]:
def AWGN(image, mean=0, sigma=25):
    """Add Additive White Gaussian Noise to an image (uint8 0–255)."""
    image = image.astype(np.float32)
    gauss = np.random.normal(mean, sigma, image.shape).astype(np.float32)
    noisy = image + gauss
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def impulse_noise(image, prob=0.05):
    """Add Impulse Noise (Salt and Pepper Noise) to an image."""
    noisy_image = image.copy()
    black = 0
    white = 255
    probs = np.random.rand(*image.shape)
    noisy_image[probs < (prob / 2)] = black
    noisy_image[probs > 1 - (prob / 2)] = white
    return noisy_image

In [46]:
import os
import matplotlib.pyplot as plt
# add noise to image and save them to respective folders (AWGN and Impulse Noise)
def add_noise_and_save(input_image_path, awgn_folder, impulse_folder):
    if not os.path.exists(awgn_folder):
        os.makedirs(awgn_folder)
    if not os.path.exists(impulse_folder):
        os.makedirs(impulse_folder)

    image = load_image_keep_channels(input_image_path)

    awgn_image = AWGN(image)
    impulse_image = impulse_noise(image)

    awgn_image_path = os.path.join(awgn_folder, os.path.basename(input_image_path))
    impulse_image_path = os.path.join(impulse_folder, os.path.basename(input_image_path))

    cv2.imwrite(awgn_image_path, awgn_image)
    cv2.imwrite(impulse_image_path, impulse_image)

    return awgn_image_path, impulse_image_path



In [47]:
# add noise to all the .bmp images in STI/Classic
input_folder = 'C:\\ImageProcessing\\Project_5\\original'
awgn_folder = 'C:\\ImageProcessing\\Project_5\\noisy_images\\AWGN'
impulse_folder = 'C:\\ImageProcessing\\Project_5\\noisy_images\\Impulse'
for filename in os.listdir(input_folder):
    if filename.endswith('.bmp'):
        input_image_path = os.path.join(input_folder, filename)
        add_noise_and_save(input_image_path, awgn_folder, impulse_folder)

#calculate PSNR and SSIM for noisy images
results = []
for filename in os.listdir(input_folder):
    if filename.endswith('.bmp'):
        input_image_path = os.path.join(input_folder, filename)
        awgn_image_path = os.path.join(awgn_folder, filename)
        impulse_image_path = os.path.join(impulse_folder, filename)

        psnr_awgn, ssim_awgn = compute_metrics(input_image_path, awgn_image_path)
        psnr_impulse, ssim_impulse = compute_metrics(input_image_path, impulse_image_path)

        results.append((filename, psnr_awgn, ssim_awgn, psnr_impulse, ssim_impulse))
# Print results in tabular format
print(f"{'Image':<20} {'PSNR_AWGN':<15} {'SSIM_AWGN':<15} {'PSNR_Impulse':<15} {'SSIM_Impulse':<15}")
for row in results:
    print(f"{row[0]:<20} {row[1]:<15.4f} {row[2]:<15.4f} {row[3]:<15.4f} {row[4]:<15.4f}")

# Visualize one example
example_image = os.path.join(input_folder, 'boatsColor.bmp')
awgn_image_path = os.path.join(awgn_folder, 'boatsColor.bmp')
impulse_image_path = os.path.join(impulse_folder, 'boatsColor.bmp')
original = load_image_keep_channels(example_image)
awgn_image = load_image_keep_channels(awgn_image_path)
impulse_image = load_image_keep_channels(impulse_image_path)
cv2.imshow('Original Image', original)
cv2.imshow('AWGN Image', awgn_image)
cv2.imshow('Impulse Noise Image', impulse_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image                PSNR_AWGN       SSIM_AWGN       PSNR_Impulse    SSIM_Impulse   
airplane.bmp         20.3525         0.2918          17.8916         0.3511         
boats.bmp            20.3609         0.3199          18.4382         0.3505         
BoatsColor.bmp       20.5026         0.3356          18.1883         0.3685         
checkerboard.bmp     23.2040         0.2957          16.1239         0.4334         
goldhill.bmp         20.4629         0.3402          18.0102         0.3783         


In [48]:
boats_path = 'C:\\ImageProcessing\\Project_5\\original\\boats.bmp'
for i in range(1, 30, 5):
    noisy_image = AWGN(load_image_keep_channels(boats_path), sigma=i)
    noisy_image_path = f'C:\\ImageProcessing\\Project_5\\noisy_images\\AWGN\\boats_sigma_{i}.bmp'
    cv2.imwrite(noisy_image_path, noisy_image)
    psnr_val, ssim_val = compute_metrics(boats_path, noisy_image_path)
    print(f'Sigma: {i}, PSNR: {psnr_val:.2f}, SSIM: {ssim_val:.4f}')
    

Sigma: 1, PSNR: 46.88, SSIM: 0.9913
Sigma: 6, PSNR: 32.54, SSIM: 0.8003
Sigma: 11, PSNR: 27.34, SSIM: 0.5989
Sigma: 16, PSNR: 24.15, SSIM: 0.4641
Sigma: 21, PSNR: 21.84, SSIM: 0.3724
Sigma: 26, PSNR: 20.02, SSIM: 0.3082


In [49]:
import cv2

noisy_path = "C:\\ImageProcessing\\Project_5\\noisy_images\\Impulse\\checkerboard.bmp"
den_path = "C:\\ImageProcessing\\Project_5\\Checker_ChatGPT.png"
orig_path = "C:\\ImageProcessing\\Project_5\\original\\checkerboard.bmp" 

orig = cv2.imread(orig_path)      # or .bmp, .jpg, etc.
den = cv2.imread(den_path)
noisy = cv2.imread(noisy_path)

h, w = orig.shape[:2]
den_resized = cv2.resize(den, (w, h), interpolation=cv2.INTER_CUBIC)
cv2.imwrite("denoised_match.bmp", den_resized)

True

In [50]:
# invert checkerboard creation to save as BMP
den_resized = cv2.imread("denoised_match.bmp")
den_resized_inverse = cv2.bitwise_not(den_resized)
cv2.imwrite("denoised_match_inverse.bmp", den_resized_inverse)

True

In [51]:

psnr_noisy, ssim_noisy = compute_metrics(orig_path, noisy_path)
print(f'Noisy Image - PSNR: {psnr_noisy:.2f}, SSIM: {ssim_noisy:.4f}')
chat_psnr, chat_ssim = compute_metrics(orig_path, "C:\\ImageProcessing\\Project_5\\denoised_match_inverse.bmp")
print(f'ChatGPT Denoised Image - PSNR: {chat_psnr:.2f}, SSIM: {chat_ssim:.4f}')
cv2.imshow('Original Checkerboard', orig)
cv2.imshow('Noisy Checkerboard', noisy)
cv2.imshow('Denoised Checkerboard', den_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

Noisy Image - PSNR: 16.12, SSIM: 0.4334
ChatGPT Denoised Image - PSNR: 45.93, SSIM: 0.8684


In [52]:
import deepinv as dinv
import torch
import cv2
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Pretrained DRUNet from deepinv (downloads weights on first use)
drunet = dinv.models.DRUNet(pretrained="download", device=device).eval()



def DRUNet_denoise(noisy_image, sigma=0.1, device=device):
    """
    Denoise a noisy image using DRUNet.

    Parameters
    ----------
    noisy_image : np.ndarray
        HxW (gray) or HxWx3 (color), usually uint8 in [0,255] or float in [0,1].
    sigma : float
        AWGN std *in [0,1]* matching how you added noise.
        e.g. AWGN with sigma=25 on uint8 -> sigma = 25/255 ≈ 0.098.
    """

    # ----- 1) Ensure we have 3 channels -----
    if noisy_image.ndim == 2:
        # gray -> 3-channel by replication so DRUNet (RGB) can process it
        noisy_image = np.stack([noisy_image] * 3, axis=-1)
    elif noisy_image.ndim == 3 and noisy_image.shape[2] == 1:
        noisy_image = np.repeat(noisy_image, 3, axis=2)

    # If BGR from OpenCV, convert to RGB (DRUNet was trained on RGB)
    noisy_rgb = noisy_image
    if noisy_rgb.dtype == np.uint8:
        noisy_rgb = noisy_rgb.astype(np.float32) / 255.0

    # ----- 2) Numpy -> torch tensor, shape (B,C,H,W) -----
    x = torch.from_numpy(noisy_rgb).permute(2, 0, 1).unsqueeze(0).to(device)

    # ----- 3) Run DRUNet -----
    with torch.no_grad():
        # DRUNet expects sigma in [0,1] for images in [0,1]
        x_denoised = drunet(x, sigma=sigma)

    # ----- 4) Back to numpy -----
    denoised = x_denoised.squeeze(0).permute(1, 2, 0).cpu().numpy()
    denoised = np.clip(denoised, 0.0, 1.0)

    # Back to uint8 BGR for OpenCV display/saving
    denoised_uint8 = (denoised * 255.0).round().astype(np.uint8)

    # convert back to BGR if you want consistent OpenCV handling
    denoised_bgr = denoised_uint8  # if your input was RGB; if it was BGR, swap here

    return denoised_bgr




In [ ]:

sigma_norm = 25/255  # for AWGN with sigma=25 on uint8 images
img = cv2.imread("C:\\ImageProcessing\\Project_5\\noisy_images\\AWGN\\BoatsColor.bmp")
denoised = DRUNet_denoise(img, sigma=sigma_norm, device=device)
cv2.imwrite("DRUNet_denoised_BoatsColor.bmp", denoised)
psnr_value, ssim_value = (compute_metrics("C:\\ImageProcessing\\Project_5\\original\\BoatsColor.bmp", "DRUNet_denoised_BoatsColor.bmp"))
print(f"DRUNet Denoised BoatsColor.bmp - PSNR: {psnr_value:.2f}, SSIM: {ssim_value:.4f}")
cv2.imshow("Noisy", img)
cv2.imshow("DRUNet Denoised", denoised)
cv2.imshow("Original", cv2.imread("C:\\ImageProcessing\\Project_5\\original\\BoatsColor.bmp"))
cv2.waitKey(0)
cv2.destroyAllWindows()


DRUNet Denoised BoatsColor.bmp - PSNR: 32.81, SSIM: 0.8755
